<a href="https://colab.research.google.com/github/DevSchmitz/whisper_jax_on_GoogleColab/blob/main/whisper_jax_medium_gradio_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 📥 Whisper: Видео/Аудио + YouTube + ПЕРЕВОДЧИК 🌍
# 1. Установка библиотек
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q yt-dlp
!pip install -q deep-translator

import gradio as gr
import whisper
import yt_dlp
import os
from deep_translator import GoogleTranslator

# 2. Загрузка модели
print("⏳ Загрузка модели Whisper... (Ждите около 1 минуты)")
model = whisper.load_model("medium")
print("✅ Модель готова!")

# --- ФУНКЦИИ ---

def transcribe_any_file(file_path):
    if file_path is None: return "⚠️ Вы не выбрали файл!"
    filename = os.path.basename(file_path)
    print(f"Обрабатываю файл: {filename}")
    try:
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        return f"❌ Ошибка: {str(e)}"

def transcribe_youtube(url):
    if not url: return "⚠️ Вставьте ссылку!"
    print(f"Качаю с YouTube: {url}")
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'yt_temp.%(ext)s',
        'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}],
        'overwrites': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return model.transcribe("yt_temp.mp3")["text"]
    except Exception as e:
        return f"❌ Ошибка YouTube: {str(e)}"

def translate_text(text, target_lang):
    if not text:
        return "⚠️ Сначала нужно получить текст (распознать файл)."

    print(f"Перевожу на {target_lang}...")

    # Коды языков
    lang_codes = {
        "Русский": "ru",
        "Украинский": "uk",
        "Английский": "en",
        "Немецкий": "de",
        "Французский": "fr",
        "Испанский": "es"
    }

    try:
        target_code = lang_codes.get(target_lang, "ru")
        # Разбиваем текст на куски, так как переводчики не любят слишком длинные тексты за раз
        translator = GoogleTranslator(source='auto', target=target_code)

        # Если текст очень большой, рубим его (простая защита)
        chunks = [text[i:i+4000] for i in range(0, len(text), 4000)]
        translated_chunks = []

        for chunk in chunks:
            translated_chunks.append(translator.translate(chunk))

        return " ".join(translated_chunks)

    except Exception as e:
        return f"❌ Ошибка перевода: {str(e)}"

# --- ИНТЕРФЕЙС ---

with gr.Blocks(title="Whisper + Переводчик") as demo:
    gr.Markdown("## 🎙️ Супер-Комбайн: Распознавание и Перевод")

    with gr.Tabs():
        # === Вкладка 1: ФАЙЛЫ ===
        with gr.TabItem("📁 Файл (Видео/Аудио)"):
            file_input = gr.File(label="Загрузите файл", file_count="single")
            file_btn = gr.Button("1. Распознать текст", variant="primary")
            file_output = gr.Textbox(label="Оригинальный текст", show_copy_button=True, lines=8)

            # Блок перевода
            gr.Markdown("---")
            with gr.Row():
                lang_drop_1 = gr.Dropdown(choices=["Русский", "Украинский", "Английский", "Немецкий"], value="Русский", label="На какой язык?")
                trans_btn_1 = gr.Button("2. Перевести этот текст")

            trans_output_1 = gr.Textbox(label="Перевод", show_copy_button=True, lines=8)

            # Логика кнопок
            file_btn.click(transcribe_any_file, inputs=file_input, outputs=file_output)
            trans_btn_1.click(translate_text, inputs=[file_output, lang_drop_1], outputs=trans_output_1)

        # === Вкладка 2: YOUTUBE ===
        with gr.TabItem("▶️ YouTube"):
            url_input = gr.Textbox(label="Ссылка на видео", placeholder="https://youtube.com/...")
            url_btn = gr.Button("1. Скачать и Распознать", variant="primary")
            url_output = gr.Textbox(label="Оригинальный текст", show_copy_button=True, lines=8)

            # Блок перевода
            gr.Markdown("---")
            with gr.Row():
                lang_drop_2 = gr.Dropdown(choices=["Русский", "Украинский", "Английский", "Немецкий"], value="Русский", label="На какой язык?")
                trans_btn_2 = gr.Button("2. Перевести этот текст")

            trans_output_2 = gr.Textbox(label="Перевод", show_copy_button=True, lines=8)

            # Логика кнопок
            url_btn.click(transcribe_youtube, inputs=url_input, outputs=url_output)
            trans_btn_2.click(translate_text, inputs=[url_output, lang_drop_2], outputs=trans_output_2)

print("🚀 Запускаю... Нажмите на ссылку ниже (gradio.live)")
demo.launch(share=True, debug=True)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
⏳ Загрузка модели Whisper... (Ждите около 1 минуты)


100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 168MiB/s]


✅ Модель готова!
🚀 Запускаю... Нажмите на ссылку ниже (gradio.live)
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://696a3516591a9d3012.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://696a3516591a9d3012.gradio.live
